<a href="https://colab.research.google.com/github/Heedou/-/blob/main/%EA%B3%A0%EB%A0%A4%EB%8C%80_%EA%B3%BC%EC%A0%9C_nsmc2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요 라이브러리 설치

In [ ]:
!pip install transformers

In [ ]:
!pip install SentencePiece

필요한 라이브러리 불러오기

In [ ]:
import tensorflow as tf
from transformers import *
import pandas as pd
import numpy as np
import json
import os

내 구글 드라이브로 연동하기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


네이버 감성분석 데이터 다운로드

In [ ]:
!git clone https://github.com/e9t/nsmc.git

fatal: destination path 'nsmc' already exists and is not an empty directory.


다운로드 받은 nsmc 데이터 확인

In [ ]:
os.listdir('nsmc')

['code',
 '.git',
 'ratings.txt',
 'README.md',
 'ratings_test.txt',
 'synopses.json',
 'ratings_train.txt',
 'raw']

test 데이터와 train 데이터로 나누어 불러오기

In [ ]:
# train 네이버 데이터 불러오기
train_nsmc = pd.read_table('nsmc/ratings_train.txt')
train_nsmc['document'] = train_nsmc['document'].str.replace('[^ㄱ-ㅎ가-힣ㅏ-ㅣ\\s\s]', '')
train_nsmc['document'].replace('', np.nan, inplace=True)
train_nsmc = train_nsmc.dropna(how='any') #결측치 제거
train_nsmc = train_nsmc.reset_index()

In [ ]:
# test 네이버 데이터 불러오기
test_nsmc = pd.read_table('nsmc/ratings_test.txt')
test_nsmc['document'] = test_nsmc['document'].str.replace('[^ㄱ-ㅎ가-힣ㅏ-ㅣ\\s\s]', '')
test_nsmc['document'].replace('', np.nan, inplace=True)
test_nsmc = test_nsmc.dropna(how='any') #결측치 제거
test_nsmc = test_nsmc.reset_index()

데이터 확인

In [ ]:
train_nsmc

,index,id,document,label
0,0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0
3,3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
...,...,...,...,...
149181,149995,6222902,인간이 문제지 소는 뭔죄인가,0
149182,149996,8549745,평점이 너무 낮아서,1
149183,149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149184,149998,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


Kobert 사용한 모델 만들기 - kobert 토크나이저 불러오기
출처 : https://github.com/monologg/KoBERT-NER

In [ ]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

토크나이저를 이용한 전체 데이터 인코딩 - 세그먼트, 마스크, 토큰 인풋

In [ ]:
kobert_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
max_sequence_length = 64
token = kobert_tokenizer.encode('흠포스터', max_length=max_sequence_length, padding='max_length')

for index, value in enumerate(token):
    if value == 1:
        token[index] = 0

In [ ]:
tokens = []
masks = []
segments = []
labels = []
max_sequence_length = 64
for i in range(len(train_nsmc)):
  text = str(train_nsmc['document'][i])
  label = train_nsmc['label'][i]

  #token 만드는 부분
  token = kobert_tokenizer.encode(text, max_length=max_sequence_length, pad_to_max_length=True)
  for index, value in enumerate(token):
    if value == 1:
        token[index] = 0

  #mask 만드는 부분
  zeros = token.count(0)
  mask = [1]*(max_sequence_length-zeros) + [0]*zeros

  #segment 만드는 부분
  segment = [0]*max_sequence_length


  #각 리스트에 저장
  tokens.append(token)
  masks.append(mask)
  segments.append(segment)
  labels.append(label)

# numpy array로 변경
tokens = np.array(tokens)
masks = np.array(masks)
segments = np.array(segments)
labels = np.array(labels)

train_x = [tokens, masks, segments]
train_y = labels

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train_x

[array([[   2, 3093, 1698, ...,    0,    0,    0],
        [   2,  517, 7989, ...,    0,    0,    0],
        [   2, 1458, 7191, ...,    0,    0,    0],
        ...,
        [   2, 3647, 5400, ...,    0,    0,    0],
        [   2, 4483, 7467, ...,    0,    0,    0],
        [   2, 4958, 3394, ...,    0,    0,    0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [ ]:
test_tokens = []
test_masks = []
test_segments = []
test_labels = []
max_sequence_length = 64
for i in range(len(test_nsmc)):
  text = str(test_nsmc['document'][i])
  label = test_nsmc['label'][i]
  #token 만드는 부분
  token = kobert_tokenizer.encode(text, max_length=max_sequence_length,  pad_to_max_length=True)
  for index, value in enumerate(token):
    if value == 1:
        token[index] = 0

  #mask 만드는 부분
  zeros = token.count(0)
  mask = [1]*(max_sequence_length-zeros) + [0]*zeros

  #segment 만드는 부분
  segment = [0]*max_sequence_length


  #각 리스트에 저장
  test_tokens.append(token)
  test_masks.append(mask)
  test_segments.append(segment)
  test_labels.append(label)

# numpy array로 변경
test_tokens = np.array(test_tokens)
test_masks = np.array(test_masks)
test_segments = np.array(test_segments)
test_labels = np.array(test_labels)

test_x = [test_tokens, test_masks, test_segments]
test_y = test_labels

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
test_x

[array([[   2,  517, 5515, ...,    0,    0,    0],
        [   2, 2145, 6844, ...,    0,    0,    0],
        [   2, 4297, 6095, ...,    0,    0,    0],
        ...,
        [   2, 1212, 5859, ...,    0,    0,    0],
        [   2, 4069, 2420, ...,    0,    0,    0],
        [   2, 1914, 5760, ...,    0,    0,    0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [ ]:
model = TFBertModel.from_pretrained('monologg/kobert', from_pt=True)
input_tokens = tf.keras.layers.Input((max_sequence_length, ), dtype=tf.int32, name='input_word_ids')
input_masks = tf.keras.layers.Input((max_sequence_length, ), dtype=tf.int32, name='input_masks')
input_segments = tf.keras.layers.Input((max_sequence_length, ), dtype=tf.int32, name='input_segment')
output = model([input_tokens, input_masks, input_segments])
output = output[0]

All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [ ]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa
# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

#embedding = tf.keras.layers.Embedding(768,128)(output)
lstm = tf.keras.layers.LSTM(128)(output)
drop = tf.keras.layers.Dropout(0.5)(lstm)
dense = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(drop)
final_model = tf.keras.Model([input_tokens, input_masks, input_segments], dense)
final_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [ ]:
final_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 64)]         0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 64)]         0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   TFBaseModelOutputWit 92186880    input_word_ids[0][0]             
                                                                 input_masks[0][0]          

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
#model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_weights_only=True)

final_model.fit(train_x, train_y, epochs=30, shuffle=True,callbacks=[early_stopping], batch_size=64, validation_data=(test_x, test_y))

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Epoch 1/30


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


2332/2332 [==============================] - ETA: 0s - loss: 0.5147 - accuracy: 0.7296

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


2332/2332 [==============================] - 2310s 970ms/step - loss: 0.5146 - accuracy: 0.7297 - val_loss: 0.2851 - val_accuracy: 0.8802
Epoch 2/30
2332/2332 [==============================] - 2278s 977ms/step - loss: 0.2604 - accuracy: 0.8952 - val_loss: 0.2708 - val_accuracy: 0.8917
Epoch 3/30
2332/2332 [==============================] - 2278s 977ms/step - loss: 0.1895 - accuracy: 0.9289 - val_loss: 0.2774 - val_accuracy: 0.8941
Epoch 4/30
2332/2332 [==============================] - 2279s 977ms/step - loss: 0.1241 - accuracy: 0.9572 - val_loss: 0.3274 - val_accuracy: 0.8926
Epoch 5/30
2332/2332 [==============================] - 2276s 976ms/step - loss: 0.0797 - accuracy: 0.9748 - val_loss: 0.3751 - val_accuracy: 0.8928
Epoch 00005: early stopping


In [ ]:
# 제출할 데이터
submit_nsmc = pd.read_table('/content/gdrive/My Drive/Colab Notebooks/naver_sentiment/제출데이터/ko_data.csv', encoding='cp949', delimiter=',')
submit_nsmc

,Id,Sentence
0,0,정말 많이 울었던 영화입니다.
1,1,시간 낭비예요.
2,2,포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.
3,3,지금 봐도 재미있는 영화!!! 코믹과 감동!!! 그리고 요리!!!
4,4,이걸 영화로 만드는 거야?얼마나 가는지 보자.
...,...,...
11182,11182,이 영화를 커플에게 추천합니다. 영화관에 가다보면 평생 잊지 못할 추억이 하나 생길...
11183,11183,심심__ 그냥 한효주 cf
11184,11184,공감해서 눈물나는 영화. 안 보신분들이 전부 제가 울었다고 하면 의아해하실텐데 보면...
11185,11185,오토바이 신은 최고네요.


In [ ]:
submit_tokens = []
submit_masks = []
submit_segments = []
max_sequence_length = 64
for i in range(len(submit_nsmc)):
  text = str(submit_nsmc['Sentence'][i])
  #token 만드는 부분
  token = kobert_tokenizer.encode(text, max_length=max_sequence_length,  pad_to_max_length=True)
  for index, value in enumerate(token):
    if value == 1:
        token[index] = 0

  #mask 만드는 부분
  zeros = token.count(0)
  mask = [1]*(max_sequence_length-zeros) + [0]*zeros

  #segment 만드는 부분
  segment = [0]*max_sequence_length


  #각 리스트에 저장
  submit_tokens.append(token)
  submit_masks.append(mask)
  submit_segments.append(segment)

# numpy array로 변경
submit_tokens = np.array(submit_tokens)
submit_masks = np.array(submit_masks)
submit_segments = np.array(submit_segments)

submit_x = [submit_tokens, submit_masks, submit_segments]

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
preds = final_model.predict(submit_x)
preds

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [ ]:
float(preds[0])

In [ ]:
submit_nsmc['Predicted']=0
for i in range(len(preds)):
  score = float(preds[i])
  if score > 0.5:
    score = 1
  else :
    score = 0
  submit_nsmc['Predicted'][i]=score

submit_nsmc.drop(['Sentence'], axis='columns', inplace=True)
submit_nsmc

In [ ]:
submit_nsmc.to_csv('/content/gdrive/My Drive/Colab Notebooks/naver_sentiment/제출데이터/nsmc 데이터 결과2.csv', index=False)